In [ ]:
import pandas as pd
import os

# --- CONFIGURACIÓN ---
# Usamos latin-1 porque ya sabemos que tus archivos originales lo requieren
encoding_origen = 'latin-1'
encoding_destino = 'utf-8-sig' # Ideal para que Power BI reconozca tildes automáticamente

def buscar_archivo(palabra_clave):
    """Busca el archivo CSV correcto en la carpeta"""
    archivos = os.listdir()
    for archivo in archivos:
        if palabra_clave in archivo and archivo.endswith(".csv") and "clean" not in archivo:
            return archivo
    return None

